In [216]:
import numpy as np
import cv2
import open3d as o3d
import matplotlib.pyplot as plt
import open3d as o3
import math
from sklearn.neighbors import NearestNeighbors



def rx(r):
    r = r/180*math.pi
    m= np.array(
    [
        [1,0,0,0],
        [0,math.cos(r),-math.sin(r),0],
        [0,math.sin(r),math.cos(r),0],
        [0,0,0,1]
    ])
    return m
def ry(r):
    r = r/180*math.pi
    m= np.array(
    [
        [math.cos(r),0,math.sin(r),0],
        [0,1,0,0],
        [-math.sin(r),0,math.cos(r),0],
        [0,0,0,1]
    ])
    return m
def rz(r):
    r = r/180*math.pi
    m= np.array(
    [
        [math.cos(r),-math.sin(r),0,0],
        [math.sin(r),math.cos(r),0,0],
        [0,0,1,0],
        [0,0,0,1]
    ])
    return m
def tran(tx,ty,tz):
    
    m= np.array(
    [
        [1,0,0,tx],
        [0,1,0,ty],
        [0,0,1,tz],
        [0,0,0,1]
    ])
    return m
M = rx(30)@ry(30)@rz(30)@tran(2,0,0)
M

array([[ 0.75      , -0.4330127 ,  0.5       ,  1.5       ],
       [ 0.64951905,  0.625     , -0.4330127 ,  1.29903811],
       [-0.125     ,  0.64951905,  0.75      , -0.25      ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [165]:
class match():
    def __init__(self,i):
        self.queryIdx = i
        self.trainIdx = i
        self.distance = 1

good_match=[]
for i in range(s.shape[0]):
    good_match.append(match(i))

In [111]:
def gause(x):
    y=1/math.sqrt(2*math.pi)*np.exp(-((x/50)**2)/2)
    return y
def affinM(t1_positions,t2_positions,good_matches):
    t1_3d_points=[]
    t2_3d_points=[]
    confidence = []
    for match in good_matches:
        t1_3d_points.append(t1_positions[match.queryIdx])
        t2_3d_points.append(t2_positions[match.trainIdx])
        c = gause(match.distance)
        confidence.append([c,c,c])
    t1_3d_points=np.array(t1_3d_points)
    t2_3d_points=np.array(t2_3d_points)
    confidence=np.array(confidence)
    Y0 =np.sum(t1_3d_points*confidence,axis=0)/np.sum(confidence,axis=0)
    X0 =np.sum(t2_3d_points*confidence,axis=0)/np.sum(confidence,axis=0)
    H = (t1_3d_points-Y0).T@(t2_3d_points-X0)
    U, s, V = np.linalg.svd(H, full_matrices=False)
    
    R=(V.T)@(U.T)
    T=X0-R@Y0
    M=np.array([
        [R[0][0],R[0][1],R[0][2],T[0]],
        [R[1][0],R[1][1],R[1][2],T[1]],
        [R[2][0],R[2][1],R[2][2],T[2]],
        [0,0,0,1]
    ])

    return M
print(M)
testM = affinM(s,g,good_match)
print(testM)

[[ 0.75       -0.4330127   0.5         1.5       ]
 [ 0.64951905  0.625      -0.4330127   1.29903811]
 [-0.125       0.64951905  0.75       -0.25      ]
 [ 0.          0.          0.          1.        ]]
[[ 0.75       -0.4330127   0.5         1.5       ]
 [ 0.64951905  0.625      -0.4330127   1.29903811]
 [-0.125       0.64951905  0.75       -0.25      ]
 [ 0.          0.          0.          1.        ]]


In [703]:
def gause(x):
    y=1/math.sqrt(2*math.pi)*np.exp(-((x/50)**2)/2)
    return y
def affinM(t1_positions,t2_positions,good_matches):
    t1_3d_points=[]
    t2_3d_points=[]
    confidence = []
    for match in good_matches:
        t1_3d_points.append(t1_positions[match.queryIdx])
        t2_3d_points.append(t2_positions[match.trainIdx])
    t1_3d_points=np.array(t1_3d_points)
    t2_3d_points=np.array(t2_3d_points)
    Y0 =np.mean(t1_3d_points,axis=0)
    X0 =np.mean(t2_3d_points,axis=0)
    H = (t1_3d_points-Y0).T@(t2_3d_points-X0)
    U, s, V = np.linalg.svd(H, full_matrices=False)
    
    R=U@V
    T=Y0-R@X0
    M=np.array([
        [R[0][0],R[0][1],R[0][2],T[0]],
        [R[1][0],R[1][1],R[1][2],T[1]],
        [R[2][0],R[2][1],R[2][2],T[2]],
        [0,0,0,1]
    ])

    return M

In [789]:
def fit_affinM(positions,M):
    homo_points=np.concatenate([positions,np.ones((positions.shape[0],1))],1).T
    newp=(M@homo_points).T[:,:-1]
    return newp
s = np.array([[0,0,0],[1,0,0]])
sc = np.tile([0,0,1],[s.shape[0],1])
g = fit_affinM(s,M)
gc = np.tile([0,1,0],[g.shape[0],1])

In [167]:
test = fit_affinM(g,testM)
testc = np.tile([1,0,0],[g.shape[0],1])

In [169]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(s)
pcd.colors = o3d.utility.Vector3dVector(sc)
pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(g)
pcd2.colors = o3d.utility.Vector3dVector(gc)
pcd3 = o3d.geometry.PointCloud()
pcd3.points = o3d.utility.Vector3dVector(test)
pcd3.colors = o3d.utility.Vector3dVector(testc)
o3d.visualization.draw_geometries([pcd,pcd2,pcd3])

In [808]:
t1=np.load("./frame/200/pos_des_kp_frame.npz")
t2=np.load("./frame/215/pos_des_kp_frame.npz")

t1.files

['positions', 'right_descriptions', 'right_kp', 'right_frame']

In [177]:
pos = t1["positions"]
aff_pos = fit_affinM(pos,M)
good_match=[]
for i in range(pos.shape[0]):
    good_match.append(match(i))

In [187]:
small_M = affinM(pos[:4],aff_pos[:4],good_match[:4])
small_M

array([[ 7.50000000e-01,  6.49519053e-01, -1.25000000e-01,
        -2.00000000e+00],
       [-4.33012702e-01,  6.25000000e-01,  6.49519053e-01,
        -5.68434189e-14],
       [ 5.00000000e-01, -4.33012702e-01,  7.50000000e-01,
         1.17950094e-12],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [190]:
pre_M = affinM(pos,aff_pos,good_match)
pre_M

array([[ 7.50000000e-01,  6.49519053e-01, -1.25000000e-01,
        -2.00000000e+00],
       [-4.33012702e-01,  6.25000000e-01,  6.49519053e-01,
        -1.70530257e-13],
       [ 5.00000000e-01, -4.33012702e-01,  7.50000000e-01,
        -1.70530257e-13],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [181]:
pos

array([[-88.20661491, 109.7467282 , 123.55867257],
       [-88.36861296, 101.12298732, 124.70296628],
       [-87.54047146, 110.95705987, 123.77072794],
       ...,
       [-35.63788644,  58.3494868 , 136.71102087],
       [-45.28952021,  29.58023817, 133.81431703],
       [-39.49082921,  46.12223734, 133.83983607]])

In [214]:
pre_pos= fit_affinM(aff_pos,pre_M)
pre_pos

array([[-88.20661491, 109.7467282 , 123.55867257],
       [-88.36861296, 101.12298732, 124.70296628],
       [-87.54047146, 110.95705987, 123.77072794],
       ...,
       [-35.63788644,  58.3494868 , 136.71102087],
       [-45.28952021,  29.58023817, 133.81431703],
       [-39.49082921,  46.12223734, 133.83983607]])

In [459]:
nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(pos)
distances, indices = nbrs.kneighbors(t2_pos)
np.count_nonzero(distances < 1)

3

In [775]:
t1_pos = t1["positions"]
t2_pos = t2["positions"]

In [793]:
def match_by_frame(t1_descriptions,t2_descriptions):
    bf = cv2.BFMatcher(crossCheck=True)
    matches = bf.match(t1_descriptions,t2_descriptions)
    good_matches = sorted(matches, key=lambda x: x.distance)
    return good_matches

In [809]:
%%time
tryout = 30
best_count = 0
bestM = None
bestDistances = None

t1_pos = t1["positions"]
t2_pos = t2["positions"]

good_matches = match_by_frame(t1["right_descriptions"],t2["right_descriptions"])

matched_t1 = np.asarray([t1_pos[match.queryIdx] for match in good_matches])
matched_t2 = np.asarray([t2_pos[match.trainIdx] for match in good_matches])

# nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(matched_t1)
# distances, indices = nbrs.kneighbors(pre_t2_pos)

for i in range(tryout):
    small_good_matches = np.random.choice(good_matches,5)
    M = affinM(t1_pos,t2_pos,small_good_matches)
    
    pre_matched_t2 = fit_affinM(matched_t2,M)
    
    distances = np.linalg.norm(matched_t1-pre_matched_t2,axis=1)
    new_count = np.count_nonzero(distances < 10)
    
    if new_count > best_count:
        best_count = new_count
        bestM = M
        bestDistances = distances
bestM,best_count

Wall time: 190 ms


(array([[  0.99717695,  -0.03175663,   0.06804157,   3.13225996],
        [  0.0257041 ,   0.99578416,   0.08805228, -14.23193055],
        [ -0.07055096,  -0.08605476,   0.99378928,  -7.82024152],
        [  0.        ,   0.        ,   0.        ,   1.        ]]),
 831)

In [810]:
best_matches = []
for i in range(len(bestDistances)):
    if bestDistances[i]<30:
        best_matches.append(good_matches[i])
pre_M = affinM(t1_pos,t2_pos,best_matches)

In [811]:
def checkM(t1_pos,t2_pos,newp):
    cpcd = np.concatenate([t1_pos,t2_pos,newp])
    g = np.tile([0,1,0],[t1_pos.shape[0],1])
    b = np.tile([0,0,1],[t2_pos.shape[0],1])
    r = np.tile([1,0,0],[newp.shape[0],1])
    cpcdc =   np.concatenate([g,b,r])
    checkpcd = o3d.geometry.PointCloud()
    checkpcd.points = o3d.utility.Vector3dVector(cpcd)
    checkpcd.colors = o3d.utility.Vector3dVector(cpcdc)
    o3d.io.write_point_cloud("checkpcd.ply", checkpcd )
newp = fit_affinM(t2_pos,pre_M)
checkM(t1_pos,t2_pos,newp)

In [760]:
bestDistances

array([ 5.24436426,  2.92540868,  1.55489484, ...,  4.50166217,
       33.47167785,  8.67571253])

In [715]:
small_good_matches = np.random.choice(good_matches,5)
M = affinM(t1_pos,t2_pos,small_good_matches)

pre_matched_t2 = fit_affinM(matched_t2,M)

distances = np.linalg.norm(matched_t1-pre_matched_t2,axis=1)
new_count = np.count_nonzero(distances < 1)
small_good_matches,M,distances,new_count

(array([<DMatch 000002480B879EF0>, <DMatch 000002480B1A2670>,
        <DMatch 000002480B956A70>, <DMatch 000002480B8CE650>,
        <DMatch 000002480B8794B0>], dtype=object),
 array([[ 0.99134941,  0.08727438,  0.09802822, -7.05295207],
        [-0.07664947,  0.99126017, -0.10736915, 29.79106698],
        [-0.10654204,  0.09892653,  0.98937472, 12.38245402],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([  0.91598816,   0.50227872,  11.42639338, ...,  35.25445635,
         96.96443714, 139.42836289]),
 19)

In [550]:
a = np.array([[2,2,3],[1,1,1],[1,2,3]])
b = np.array([[1,2,3],[1,2,3],[1,2,3]])
np.linalg.norm(a-b, axis=1)

array([1.        , 2.23606798, 0.        ])

In [812]:
p = 0.99999 # 外れ値を含まない確率
w = 0.8 # inlier/全サンプル数
n = 5 # サンプリング数
# iteration k
# p = 1 - ( 1 - w ** n ) ** k
k = math.log(1-p)/math.log(1-w**n)
print(f"すべてのサンプリングで\n外れ値が含まれない確率が\n{p}\nとなる\nイテレーション回数は\n{k}回\n")


すべてのサンプリングで
外れ値が含まれない確率が
0.99999
となる
イテレーション回数は
28.99828846911848回



In [509]:
bestDistances,bestIndices,bestM,best_count

(array([[ 3.48390769],
        [ 1.57811415],
        [ 5.21113375],
        ...,
        [14.60011191],
        [ 4.33384777],
        [ 1.51313697]]),
 array([[  44],
        [  51],
        [2187],
        ...,
        [1131],
        [1258],
        [1952]], dtype=int64),
 array([[ 0.99177077,  0.07524319,  0.10358185, -7.28611784],
        [-0.06106097,  0.98911685, -0.1338634 , 33.92877709],
        [-0.11252686,  0.126437  ,  0.98557161, 12.38475841],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 2064)

In [519]:
x = np.array([[1, 2,3],[1, 2,3],[1, 2,3]])
y =  np.array([[3, 2,3],[1, 2,3],[1, 2,3]])
np.linalg.norm(x-y)

2.0